### Preprocess `scanned objects` files 
Put this notebook under
```
./models
```
e.g., where you copied 
```
./mujoco_scanned_objects/models/2_of_Jenga_Classic_Game
./mujoco_scanned_objects/models/3D_Dollhouse_Happy_Brother
...
```

In [1]:
import os
import shutil
import glob
import xml.etree.ElementTree as ET
print ("Ready.")

Ready.


In [2]:
# base asset paths
asset_path     = './models'
processed_path = './models_processed'
mesh_path      = './mesh'
print ("Ready.")

Ready.


#### Remove existing folders ('processed_path' and 'mesh_path')

In [3]:
if os.path.exists(processed_path):
    shutil.rmtree(processed_path)
    print("[%s] removed."%(processed_path))
if os.path.exists(mesh_path):
    shutil.rmtree(mesh_path)
    print("[%s] removed."%(mesh_path))
print ("Done.")

Done.


#### Create folders

In [4]:
# create processed folder if it does not exist
os.makedirs(processed_path, exist_ok=True)
print ("[%s] created."%(processed_path))
# create mesh folder in processed path
os.makedirs(mesh_path, exist_ok=True)
print ("[%s] created."%(mesh_path))
print ("Ready.")

[./models_processed] created.
[./mesh] created.
Ready.


#### Main loop
```
위의 코드는 다음과 같은 작업을 수행합니다:

1. `asset_path`의 각 폴더를 순회합니다.
2. 각 폴더에 대해 `processed_path`에 해당 폴더를 생성합니다.
3. 폴더 이름이 숫자로 시작하면 파일 이름에 'model_' 접두사를 추가합니다.

세부 작업 단계는 다음과 같습니다:

1. `model.xml` 파일을 `processed_path/폴더이름.xml`로 복사합니다.
2. `model_collision_XX.obj` 파일을 `mesh_path/폴더이름_collision_XX.obj`로 복사합니다.
3. `texture.png` 파일을 `mesh_path/폴더이름.png`로 복사합니다.
4. `model.obj` 파일을 `processed_path/폴더이름.obj`로 복사합니다.
5. `processed_path/폴더이름.obj` 파일을 `mesh_path/폴더이름.obj`로 복사합니다.
6. `processed_path/폴더이름.xml` 파일의 내용을 수정합니다:
   - "model"을 "폴더이름"으로 변경합니다.
   - "texture"를 "폴더이름"으로 변경합니다.
   - "material_"을 "material_폴더이름_"로 변경합니다.
   - `<worldbody>` 내부의 첫 번째 `<body>` 요소 뒤에 `<freejoint />`를 추가합니다.

마지막으로 모든 작업이 완료되면 "Done." 메시지를 출력합니다.
```

In [5]:
# iterate through each folder in the asset_path
verbose_detail = False
for folder_name in os.listdir(asset_path):
    folder_path = os.path.join(asset_path, folder_name)
    
    if os.path.isdir(folder_path): 
        print ("\nfolder_name: [%s]"%(folder_name))
        
        # create corresponding folder in processed_path
        processed_folder_path = os.path.join(processed_path, folder_name)
        os.makedirs(processed_folder_path, exist_ok=True)
        
        # Prefix for files if folder_name starts with a digit
        prefix = 'model_' if folder_name[0].isdigit() else ''
        """
            Step 1: 
            # Copy and rename 'model.xml' to 'processed_path/folder_name.xml'
            Copy and rename 'model.xml' to 'processed_path/model.xml'
        """
        xml_file = os.path.join(folder_path, 'model.xml')
        # new_xml_file = os.path.join(processed_folder_path, f'{prefix}{folder_name}.xml')
        new_xml_file = os.path.join(processed_folder_path, f'model.xml')
        if os.path.exists(xml_file):
            shutil.copy(xml_file, new_xml_file)
            if verbose_detail: print(" Copy [%s] to [%s]"%(xml_file,new_xml_file))

        """
            Step 2: 
            Copy and rename 'model_collision_XX.obj' to 'mesh_path/folder_name_collision_XX.obj'
        """
        obj_files = glob.glob(os.path.join(folder_path, 'model_collision_*.obj'))
        for obj_idx,obj_file in enumerate(obj_files):
            obj_file_name = os.path.basename(obj_file)
            new_obj_file_name = obj_file_name.replace('model', f'{prefix}{folder_name}')
            new_obj_file = os.path.join(mesh_path, new_obj_file_name)
            shutil.copy(obj_file, new_obj_file)
            if verbose_detail: print("  [%d] Copy [%s] to [%s]"%(obj_idx,obj_file,new_obj_file))

        """
            Step 3: 
            Copy and rename 'texture.png' to 'mesh_path/folder_name.png'
        """
        texture_file = os.path.join(folder_path, 'texture.png')
        new_texture_file = os.path.join(mesh_path, f'{prefix}{folder_name}.png')
        if os.path.exists(texture_file):
            shutil.copy(texture_file, new_texture_file)
            if verbose_detail: print(" Copy [%s] to [%s]"%(texture_file,new_texture_file))

        """
            Step 4: 
            Copy and rename 'model.obj' to 'processed_path/folder_name.obj'
        """
        model_obj_file = os.path.join(folder_path, 'model.obj')
        new_model_obj_file = os.path.join(processed_folder_path, f'{prefix}{folder_name}.obj')
        if os.path.exists(model_obj_file):
            shutil.copy(model_obj_file, new_model_obj_file)
            if verbose_detail: print(" Copy [%s] to [%s]"%(model_obj_file,new_model_obj_file))

        """
           Step 5:
           Copy 'processed_path/folder_name.obj' to 'mesh_path/folder_name.obj'
        """
        if os.path.exists(new_model_obj_file):
            shutil.copy(new_model_obj_file, mesh_path)
            if verbose_detail: print(" Copy [%s] to [%s]"%(new_model_obj_file,mesh_path))

        """ 
            Step 6: 
            Modify the contents of 'processed_path/folder_name.xml'
             - "model" -> "folder_name"
             - "texture" -> "folder_name"
             - "material_" -> "folder_name_"
             - Add <freejoint/>
        """
        if os.path.exists(new_xml_file):
            tree = ET.parse(new_xml_file)
            root = tree.getroot()
            
            for elem in root.iter():
                for key, value in elem.attrib.items():
                    if 'model' in value:
                        new_value = value.replace('model', f'{prefix}{folder_name}')
                        elem.set(key, new_value)
                    if 'texture' in value:
                        new_value = value.replace('texture', f'{prefix}{folder_name}')
                        elem.set(key, new_value)
                    if 'material_' in value:
                        new_value = value.replace('material_', f'material_{folder_name}_')
                        elem.set(key, new_value)
                
                if elem.text:
                    if 'model' in elem.text:
                        new_text = elem.text.replace('model', f'{prefix}{folder_name}')
                        elem.text = new_text
                    if 'texture' in elem.text:
                        new_text = elem.text.replace('texture', f'{prefix}{folder_name}')
                        elem.text = new_text
                    if 'material_' in elem.text:
                        new_text = elem.text.replace('material_', f'material_{folder_name}_')
                        elem.text = new_text
            
            # Add <freejoint /> after the first <body> inside <worldbody>
            for worldbody in root.iter('worldbody'):
                first_body = worldbody.find('body')
                if first_body is not None:
                    freejoint = ET.Element('freejoint')
                    first_body.insert(1, freejoint)
            
            tree.write(new_xml_file)
            print(" Saved [%s]"%(new_xml_file))

print ("Done.")


folder_name: [MK7]
 Saved [./models_processed/MK7/model.xml]

folder_name: [GARDEN_SWING]
 Saved [./models_processed/GARDEN_SWING/model.xml]

folder_name: [JUNGLE_HEIGHT]
 Saved [./models_processed/JUNGLE_HEIGHT/model.xml]

folder_name: [STEAK_SET]
 Saved [./models_processed/STEAK_SET/model.xml]

folder_name: [BIRD_RATTLE]
 Saved [./models_processed/BIRD_RATTLE/model.xml]

folder_name: [Android_Figure_Chrome]
 Saved [./models_processed/Android_Figure_Chrome/model.xml]

folder_name: [MOSAIC]
 Saved [./models_processed/MOSAIC/model.xml]

folder_name: [TZX_Runner]
 Saved [./models_processed/TZX_Runner/model.xml]

folder_name: [DRAGON_W]
 Saved [./models_processed/DRAGON_W/model.xml]

folder_name: [Racoon]
 Saved [./models_processed/Racoon/model.xml]

folder_name: [MEAT_SET]
 Saved [./models_processed/MEAT_SET/model.xml]

folder_name: [BATHROOM_CLASSIC]
 Saved [./models_processed/BATHROOM_CLASSIC/model.xml]

folder_name: [MINI_FIRE_ENGINE]
 Saved [./models_processed/MINI_FIRE_ENGINE/model